# Tugas 1 : Basic NLP Tools
Oleh : 
- Safiq Faray (13519145)

In [85]:
import spacy
nlp = spacy.load("en_core_web_sm")
text = nlp("Hello bro, i live in Indonesia and i love lasagna. Also, i love burgers and pizzas. Man, i wish i could get some pizza right now. I worked at itb")

## Sentence Splitter

In [86]:
def sentence_split(text):
    return list(text.sents)
sentence_split(text)

[Hello bro, i live in Indonesia and i love lasagna.,
 Also, i love burgers and pizzas.,
 Man, i wish i could get some pizza right now.,
 I worked at itb]

## Tokenization

In [87]:
def tokenization(text):
    return [token.text for token in text]
tokenization(text)

['Hello',
 'bro',
 ',',
 'i',
 'live',
 'in',
 'Indonesia',
 'and',
 'i',
 'love',
 'lasagna',
 '.',
 'Also',
 ',',
 'i',
 'love',
 'burgers',
 'and',
 'pizzas',
 '.',
 'Man',
 ',',
 'i',
 'wish',
 'i',
 'could',
 'get',
 'some',
 'pizza',
 'right',
 'now',
 '.',
 'I',
 'worked',
 'at',
 'itb']

## Stemming

In [88]:
from nltk.stem import PorterStemmer
def stemming(token):
    stemmer = PorterStemmer()
    return stemmer.stem(token.text)
tes = nlp("writing")
stemming(tes[0])

'write'

## Lemmatization

In [89]:
def lemmatization(token):
    return token.lemma_
lemmatization(tes[0])

'write'

## Entity Masking

In [90]:
def mask_entity(text):
    final_text = ' '.join(token.text if token.ent_type_ == "" else "MASK" for token in text)
    return final_text
mask_entity(text)

'Hello bro , i live in MASK and i love lasagna . Also , i love burgers and pizzas . Man , i wish i could get some pizza right now . I worked at MASK'

## Entity Masking Custom Word

In [93]:
from enum import Enum
from spacy.tokens import Span
EntityType = Enum('EntityType',['Person', 'Norp', 'Fac', 'Org', 'Gpe', 'Loc', 'Product', 'Event', 'Work_of_art', 'Law', 'Language','Date', 'Time', 'Percent', 'Money', 'Quantity', 'Ordinal', 'Cardinal'])

def get_token_index(text, word):
    idx_tokens = []
    idx_begin = 0
    for token in text:
        if token.text == word:
            idx_end = idx_begin+1
            idx_tokens.append([idx_begin, idx_end])
        idx_begin+=1
    return idx_tokens

def check_unique_entities(new_ents, original_ents):
    flag = True
    for ent in new_ents:
        flag = flag and (ent not in original_ents)
    return flag

def create_new_entity(text, entity_word, entity_type: EntityType):
    idx_tokens = get_token_index(text, entity_word)
    if (len(idx_tokens) == 0):
        raise Exception('Custom word to be defined as entity not found in text')
    new_entities = []
    org_ents = list(text.ents)
    for it in idx_tokens:
        new_entity = Span(text, it[0], it[1], label=entity_type.name.upper())
        new_entities.append(new_entity)
    if check_unique_entities(new_entities, org_ents):
        text.ents = org_ents + new_entities

def mask_entity_custom(text, entity_word, entity_type: EntityType):
    for ent in entity_word:
        create_new_entity(text, ent, entity_type)
    return mask_entity(text)

mask_entity_custom(text, ["bro"], EntityType.Law)


'Hello MASK , i live in MASK and i love lasagna . Also , i love burgers and pizzas . Man , i wish i could get some pizza right now . I worked at MASK'